In [10]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import pickle

# Hardcoded absolute path to dataset (update if needed)
DATA_PATH = r"C:\Users\Roshan\OneDrive\Desktop\customer-churn-retention\data\WA_Fn-UseC_-Telco-Customer-Churn.csv"

# Model directory and path
MODEL_DIR = r"C:\Users\Roshan\OneDrive\Desktop\customer-churn-retention\models"
MODEL_PATH = os.path.join(MODEL_DIR, 'model.pkl')

# Create models dir if not exists
os.makedirs(MODEL_DIR, exist_ok=True)

# Load dataset
df = pd.read_csv(DATA_PATH)

# Preprocessing
df.drop('customerID', axis=1, inplace=True)

df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'].fillna(df['TotalCharges'].median(), inplace=True)

df['Churn'] = df['Churn'].map({'Yes':1, 'No':0})

# One-hot encoding categorical variables
X = pd.get_dummies(df.drop('Churn', axis=1), drop_first=True)
y = df['Churn']

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# Train model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

# Save model and feature columns
with open(MODEL_PATH, 'wb') as f:
    pickle.dump({'model': model, 'columns': X.columns.tolist()}, f)

print(f"Model saved to {MODEL_PATH}")


C:\Users\Roshan\AppData\Local\Temp\ipykernel_18280\1403826730.py:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['TotalCharges'].fillna(df['TotalCharges'].median(), inplace=True)


              precision    recall  f1-score   support

           0       0.83      0.89      0.86      1035
           1       0.62      0.49      0.55       374

    accuracy                           0.79      1409
   macro avg       0.73      0.69      0.71      1409
weighted avg       0.77      0.79      0.78      1409

Model saved to C:\Users\Roshan\OneDrive\Desktop\customer-churn-retention\models\model.pkl


In [11]:
import os
from flask import Flask, request, jsonify
import pickle
import pandas as pd

app = Flask(__name__)

# Absolute path to the model file
MODEL_PATH = r"C:\Users\Roshan\OneDrive\Desktop\customer-churn-retention\models\model.pkl"

# Load model and columns
with open(MODEL_PATH, 'rb') as f:
    data = pickle.load(f)

model = data['model']
model_columns = data['columns']

@app.route('/')
def home():
    return "Customer Churn Prediction API is running."

@app.route('/predict', methods=['POST'])
def predict():
    json_data = request.get_json(force=True)
    input_df = pd.DataFrame([json_data])

    # Handle numeric conversion & missing data
    if 'TotalCharges' in input_df.columns:
        input_df['TotalCharges'] = pd.to_numeric(input_df['TotalCharges'], errors='coerce')
        input_df['TotalCharges'].fillna(0, inplace=True)

    # One-hot encode input
    input_encoded = pd.get_dummies(input_df)

    # Add missing columns
    for col in model_columns:
        if col not in input_encoded.columns:
            input_encoded[col] = 0

    # Ensure columns order
    input_encoded = input_encoded[model_columns]

    pred = model.predict(input_encoded)[0]
    proba = model.predict_proba(input_encoded)[0][1]

    retention_advice = "Offer 10% discount to retain customer." if pred == 1 else "No retention needed."

    return jsonify({
        'churn_prediction': int(pred),
        'churn_probability': float(proba),
        'retention_advice': retention_advice
    })

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


C:\Users\Roshan\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.7.1 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\Roshan\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.7.1 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\Roshan\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [13]:
df['TotalCharges'] = df['TotalCharges'].fillna(df['TotalCharges'].median())


In [14]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'] = df['TotalCharges'].fillna(df['TotalCharges'].median())


In [15]:
{
  "gender": "Female",
  "SeniorCitizen": 0,
  "Partner": "Yes",
  "Dependents": "No",
  "tenure": 12,
  "PhoneService": "Yes",
  "MultipleLines": "No",
  "InternetService": "Fiber optic",
  "OnlineSecurity": "No",
  "OnlineBackup": "Yes",
  "DeviceProtection": "No",
  "TechSupport": "No",
  "StreamingTV": "No",
  "StreamingMovies": "No",
  "Contract": "Month-to-month",
  "PaperlessBilling": "Yes",
  "PaymentMethod": "Electronic check",
  "MonthlyCharges": 70.35,
  "TotalCharges": 1397.475
}


{'gender': 'Female',
 'SeniorCitizen': 0,
 'Partner': 'Yes',
 'Dependents': 'No',
 'tenure': 12,
 'PhoneService': 'Yes',
 'MultipleLines': 'No',
 'InternetService': 'Fiber optic',
 'OnlineSecurity': 'No',
 'OnlineBackup': 'Yes',
 'DeviceProtection': 'No',
 'TechSupport': 'No',
 'StreamingTV': 'No',
 'StreamingMovies': 'No',
 'Contract': 'Month-to-month',
 'PaperlessBilling': 'Yes',
 'PaymentMethod': 'Electronic check',
 'MonthlyCharges': 70.35,
 'TotalCharges': 1397.475}

In [17]:
import streamlit as st
import pandas as pd
import pickle
import requests

# Load model column info
MODEL_PATH = r"C:\Users\Roshan\OneDrive\Desktop\customer-churn-retention\models\model.pkl"
with open(MODEL_PATH, 'rb') as f:
    model_data = pickle.load(f)

model_columns = model_data['columns']

st.title("📉 Customer Churn Prediction")
st.write("Enter customer details below to predict churn and get retention advice.")

# Input fields
gender = st.selectbox("Gender", ["Male", "Female"])
senior = st.selectbox("Senior Citizen", [0, 1])
partner = st.selectbox("Partner", ["Yes", "No"])
dependents = st.selectbox("Dependents", ["Yes", "No"])
tenure = st.number_input("Tenure (months)", min_value=0, max_value=100, value=12)
phone_service = st.selectbox("Phone Service", ["Yes", "No"])
multiple_lines = st.selectbox("Multiple Lines", ["Yes", "No", "No phone service"])
internet_service = st.selectbox("Internet Service", ["DSL", "Fiber optic", "No"])
online_security = st.selectbox("Online Security", ["Yes", "No", "No internet service"])
online_backup = st.selectbox("Online Backup", ["Yes", "No", "No internet service"])
device_protection = st.selectbox("Device Protection", ["Yes", "No", "No internet service"])
tech_support = st.selectbox("Tech Support", ["Yes", "No", "No internet service"])
streaming_tv = st.selectbox("Streaming TV", ["Yes", "No", "No internet service"])
streaming_movies = st.selectbox("Streaming Movies", ["Yes", "No", "No internet service"])
contract = st.selectbox("Contract", ["Month-to-month", "One year", "Two year"])
paperless_billing = st.selectbox("Paperless Billing", ["Yes", "No"])
payment_method = st.selectbox("Payment Method", [
    "Electronic check", "Mailed check", "Bank transfer (automatic)", "Credit card (automatic)"
])
monthly_charges = st.number_input("Monthly Charges", min_value=0.0, max_value=1000.0, value=70.35)
total_charges = st.number_input("Total Charges", min_value=0.0, max_value=10000.0, value=1397.47)

# When button is clicked
if st.button("Predict Churn"):
    customer_data = {
        "gender": gender,
        "SeniorCitizen": senior,
        "Partner": partner,
        "Dependents": dependents,
        "tenure": tenure,
        "PhoneService": phone_service,
        "MultipleLines": multiple_lines,
        "InternetService": internet_service,
        "OnlineSecurity": online_security,
        "OnlineBackup": online_backup,
        "DeviceProtection": device_protection,
        "TechSupport": tech_support,
        "StreamingTV": streaming_tv,
        "StreamingMovies": streaming_movies,
        "Contract": contract,
        "PaperlessBilling": paperless_billing,
        "PaymentMethod": payment_method,
        "MonthlyCharges": monthly_charges,
        "TotalCharges": total_charges
    }

    # Send POST request to Flask API
    response = requests.post("http://127.0.0.1:5000/predict", json=customer_data)

    if response.status_code == 200:
        result = response.json()
        st.success(f"Churn Prediction: {'Yes' if result['churn_prediction'] == 1 else 'No'}")
        st.info(f"Probability of churn: {result['churn_probability'] * 100:.2f}%")
        st.warning(f"Retention advice: {result['retention_advice']}")
    else:
        st.error("Error in prediction. Check if Flask API is running.")

C:\Users\Roshan\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.7.1 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\Roshan\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.7.1 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [18]:
from flask import Flask, request, jsonify
import pickle
import pandas as pd
import numpy as np
import os

app = Flask(__name__)

# Load the model and columns info once
MODEL_PATH = r"C:\Users\Roshan\OneDrive\Desktop\customer-churn-retention\models\model.pkl"
with open(MODEL_PATH, 'rb') as f:
    model_data = pickle.load(f)

model = model_data['model']
model_columns = model_data['columns']

def preprocess_input(data):
    # Convert input dictionary to DataFrame with one row
    df = pd.DataFrame([data])

    # We have to encode categorical variables same way as training:
    # The model was trained with get_dummies(drop_first=True)
    # So we apply get_dummies here and add missing columns with 0

    df_encoded = pd.get_dummies(df)

    # Add missing columns (that model expects) with 0
    for col in model_columns:
        if col not in df_encoded.columns:
            df_encoded[col] = 0

    # Ensure the order of columns matches training data
    df_encoded = df_encoded[model_columns]

    return df_encoded

@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.get_json()
        # Preprocess
        X = preprocess_input(data)

        # Predict probability and class
        prob = model.predict_proba(X)[:, 1][0]
        pred = int(model.predict(X)[0])

        # Simple retention advice logic
        advice = "Offer discount or personalized plan" if prob > 0.5 else "No immediate action needed"

        # Return response as JSON
        return jsonify({
            "churn_prediction": pred,
            "churn_probability": prob,
            "retention_advice": advice
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    # Run Flask app on port 5000
    app.run(debug=True)


C:\Users\Roshan\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.7.1 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


 * Serving Flask app '__main__'
 * Debug mode: on


C:\Users\Roshan\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.7.1 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\Roshan\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [19]:
import requests

url = "http://127.0.0.1:5000/predict"

customer_data = {
    "gender": "Female",
    "SeniorCitizen": 0,
    "Partner": "Yes",
    "Dependents": "No",
    "tenure": 12,
    "PhoneService": "Yes",
    "MultipleLines": "No",
    "InternetService": "Fiber optic",
    "OnlineSecurity": "No",
    "OnlineBackup": "Yes",
    "DeviceProtection": "No",
    "TechSupport": "No",
    "StreamingTV": "No",
    "StreamingMovies": "No",
    "Contract": "Month-to-month",
    "PaperlessBilling": "Yes",
    "PaymentMethod": "Electronic check",
    "MonthlyCharges": 70.35,
    "TotalCharges": 1397.475
}

response = requests.post(url, json=customer_data)

if response.ok:
    print("Prediction Result:")
    print(response.json())
else:
    print(f"Error: {response.status_code} - {response.text}")


Prediction Result:
{'churn_prediction': 1, 'churn_probability': 0.64, 'retention_advice': 'Offer 10% discount to retain customer.'}


In [21]:
import streamlit as st
import requests

st.title("📉 Customer Churn Prediction")

# Input fields
gender = st.selectbox("Gender", ["Male", "Female"])
senior = st.selectbox("Senior Citizen", [0, 1])
partner = st.selectbox("Partner", ["Yes", "No"])
dependents = st.selectbox("Dependents", ["Yes", "No"])
tenure = st.number_input("Tenure (months)", min_value=0, max_value=100, value=12)
phone_service = st.selectbox("Phone Service", ["Yes", "No"])
multiple_lines = st.selectbox("Multiple Lines", ["Yes", "No", "No phone service"])
internet_service = st.selectbox("Internet Service", ["DSL", "Fiber optic", "No"])
online_security = st.selectbox("Online Security", ["Yes", "No", "No internet service"])
online_backup = st.selectbox("Online Backup", ["Yes", "No", "No internet service"])
device_protection = st.selectbox("Device Protection", ["Yes", "No", "No internet service"])
tech_support = st.selectbox("Tech Support", ["Yes", "No", "No internet service"])
streaming_tv = st.selectbox("Streaming TV", ["Yes", "No", "No internet service"])
streaming_movies = st.selectbox("Streaming Movies", ["Yes", "No", "No internet service"])
contract = st.selectbox("Contract", ["Month-to-month", "One year", "Two year"])
paperless_billing = st.selectbox("Paperless Billing", ["Yes", "No"])
payment_method = st.selectbox("Payment Method", [
    "Electronic check", "Mailed check", "Bank transfer (automatic)", "Credit card (automatic)"
])
monthly_charges = st.number_input("Monthly Charges", min_value=0.0, max_value=1000.0, value=70.35)
total_charges = st.number_input("Total Charges", min_value=0.0, max_value=10000.0, value=1397.47)

if st.button("Predict Churn"):
    customer_data = {
        "gender": gender,
        "SeniorCitizen": senior,
        "Partner": partner,
        "Dependents": dependents,
        "tenure": tenure,
        "PhoneService": phone_service,
        "MultipleLines": multiple_lines,
        "InternetService": internet_service,
        "OnlineSecurity": online_security,
        "OnlineBackup": online_backup,
        "DeviceProtection": device_protection,
        "TechSupport": tech_support,
        "StreamingTV": streaming_tv,
        "StreamingMovies": streaming_movies,
        "Contract": contract,
        "PaperlessBilling": paperless_billing,
        "PaymentMethod": payment_method,
        "MonthlyCharges": monthly_charges,
        "TotalCharges": total_charges
    }

    # Call Flask API
    try:
        response = requests.post("http://127.0.0.1:5000/predict", json=customer_data)
        response.raise_for_status()
        result = response.json()

        st.success(f"Churn Prediction: {'Yes' if result['churn_prediction'] == 1 else 'No'}")
        st.info(f"Probability of churn: {result['churn_probability'] * 100:.2f}%")
        st.warning(f"Retention advice: {result['retention_advice']}")
    except Exception as e:
        st.error(f"Error during prediction: {e}")

In [25]:
from IPython.display import display, HTML

html_code = """
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1" />
  <title>Customer Churn Prediction</title>
  <style>
    body {
      font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
      background: #f5f7fa;
      margin: 0;
      padding: 20px;
      display: flex;
      justify-content: center;
    }
    .container {
      background: white;
      max-width: 600px;
      width: 100%;
      border-radius: 8px;
      padding: 30px;
      box-shadow: 0 0 15px rgba(0,0,0,0.1);
    }
    h1 {
      color: #333;
      text-align: center;
    }
    label {
      display: block;
      margin-top: 15px;
      font-weight: 600;
      color: #555;
    }
    select, input[type=number] {
      width: 100%;
      padding: 10px;
      margin-top: 5px;
      border: 1px solid #ccc;
      border-radius: 5px;
      box-sizing: border-box;
      font-size: 16px;
    }
    button {
      margin-top: 25px;
      width: 100%;
      padding: 12px;
      font-size: 18px;
      background: #007bff;
      color: white;
      border: none;
      border-radius: 6px;
      cursor: pointer;
      transition: background 0.3s ease;
    }
    button:hover {
      background: #0056b3;
    }
    .result {
      margin-top: 25px;
      padding: 15px;
      border-radius: 6px;
      font-weight: bold;
      font-size: 18px;
      text-align: center;
    }
    .yes { background: #ffdddd; color: #cc0000; }
    .no { background: #ddffdd; color: #008000; }
  </style>
</head>
<body>
  <div class="container">
    <h1>Customer Churn Prediction</h1>
    <form id="churn-form">
      <label for="gender">Gender</label>
      <select id="gender" required>
        <option value="Male">Male</option>
        <option value="Female" selected>Female</option>
      </select>   
      <label for="senior">Senior Citizen</label>
      <select id="senior" required>
        <option value="0" selected>No</option>
        <option value="1">Yes</option>
      </select>

      <label for="partner">Partner</label>
      <select id="partner" required>
        <option value="Yes" selected>Yes</option>
        <option value="No">No</option>
      </select>

      <label for="dependents">Dependents</label>
      <select id="dependents" required>
        <option value="Yes">Yes</option>
        <option value="No" selected>No</option>
      </select>

      <label for="tenure">Tenure (months)</label>
      <input type="number" id="tenure" min="0" max="100" value="12" required />

      <!-- Add more fields as needed like PhoneService, InternetService, etc. -->

      <label for="monthlyCharges">Monthly Charges</label>
      <input type="number" id="monthlyCharges" min="0" step="0.01" value="70.35" required />

      <label for="totalCharges">Total Charges</label>
      <input type="number" id="totalCharges" min="0" step="0.01" value="1397.47" required />

      <button type="submit">Predict Churn</button>
    </form>

    <div id="result" class="result" style="display:none;"></div>
  </div>

  <script>
    document.getElementById('churn-form').addEventListener('submit', async function(e) {
      e.preventDefault();
      const resultDiv = document.getElementById('result');
      resultDiv.style.display = 'none';

      const data = {
        gender: document.getElementById('gender').value,
        SeniorCitizen: Number(document.getElementById('senior').value),
        Partner: document.getElementById('partner').value,
        Dependents: document.getElementById('dependents').value,
        tenure: Number(document.getElementById('tenure').value),
        MonthlyCharges: Number(document.getElementById('monthlyCharges').value),
        TotalCharges: Number(document.getElementById('totalCharges').value),
        PhoneService: "Yes",
        MultipleLines: "No",
        InternetService: "Fiber optic",
        OnlineSecurity: "No",
        OnlineBackup: "Yes",
        DeviceProtection: "No",
        TechSupport: "No",
        StreamingTV: "No",
        StreamingMovies: "No",
        Contract: "Month-to-month",
        PaperlessBilling: "Yes",
        PaymentMethod: "Electronic check"
      };

      try {
        const response = await fetch('http://127.0.0.1:5000/predict', {
          method: 'POST',
          headers: { 'Content-Type': 'application/json' },
          body: JSON.stringify(data)
        });

        if (!response.ok) {
          throw new Error('Network response was not ok');
        }

        const result = await response.json();
        let message = `Churn Prediction: ${result.churn_prediction === 1 ? 'Yes' : 'No'}<br>`;
        message += `Probability of churn: ${(result.churn_probability * 100).toFixed(2)}%<br>`;
        message += `Retention advice: ${result.retention_advice}`;

        resultDiv.innerHTML = message;
        resultDiv.className = 'result ' + (result.churn_prediction === 1 ? 'yes' : 'no');
        resultDiv.style.display = 'block';
      } catch (err) {
        resultDiv.innerHTML = "Error in prediction: " + err.message;
        resultDiv.className = 'result no';
        resultDiv.style.display = 'block';
      }
    });
  </script>
</body>
</html>
"""

display(HTML(html_code))


In [26]:
import streamlit as st

st.set_page_config(page_title="Customer Churn Prediction", layout="centered")

st.title("Customer Churn Prediction")

with st.form("churn_form"):
    gender = st.selectbox("Gender", options=["Male", "Female"], index=1)
    senior = st.selectbox("Senior Citizen", options=["No", "Yes"], index=0)
    partner = st.selectbox("Partner", options=["Yes", "No"], index=0)
    dependents = st.selectbox("Dependents", options=["Yes", "No"], index=1)
    tenure = st.number_input("Tenure (months)", min_value=0, max_value=100, value=12)
    monthly_charges = st.number_input("Monthly Charges", min_value=0.0, value=70.35, step=0.01)
    total_charges = st.number_input("Total Charges", min_value=0.0, value=1397.47, step=0.01)
    
    submitted = st.form_submit_button("Predict Churn")

if submitted:
    # Prepare data dictionary similar to your JS object
    data = {
        "gender": gender,
        "SeniorCitizen": 1 if senior == "Yes" else 0,
        "Partner": partner,
        "Dependents": dependents,
        "tenure": tenure,
        "MonthlyCharges": monthly_charges,
        "TotalCharges": total_charges,
        "PhoneService": "Yes",
        "MultipleLines": "No",
        "InternetService": "Fiber optic",
        "OnlineSecurity": "No",
        "OnlineBackup": "Yes",
        "DeviceProtection": "No",
        "TechSupport": "No",
        "StreamingTV": "No",
        "StreamingMovies": "No",
        "Contract": "Month-to-month",
        "PaperlessBilling": "Yes",
        "PaymentMethod": "Electronic check",
    }

    # Example: Replace this with your actual prediction API call or model inference
    import random
    churn_prediction = random.choice([0, 1])
    churn_probability = random.uniform(0.3, 0.9)
    retention_advice = "Offer discount" if churn_prediction == 1 else "No immediate action needed"

    # Display results with color-coded message
    if churn_prediction == 1:
        st.error(f"Churn Prediction: Yes\nProbability of churn: {churn_probability:.2%}\nRetention advice: {retention_advice}")
    else:
        st.success(f"Churn Prediction: No\nProbability of churn: {churn_probability:.2%}\nRetention advice: {retention_advice}")


In [27]:
import requests

if submitted:
    try:
        response = requests.post('http://127.0.0.1:5000/predict', json=data)
        response.raise_for_status()
        result = response.json()
        churn_prediction = result['churn_prediction']
        churn_probability = result['churn_probability']
        retention_advice = result['retention_advice']

        if churn_prediction == 1:
            st.error(f"Churn Prediction: Yes\nProbability of churn: {churn_probability:.2%}\nRetention advice: {retention_advice}")
        else:
            st.success(f"Churn Prediction: No\nProbability of churn: {churn_probability:.2%}\nRetention advice: {retention_advice}")
    except Exception as e:
        st.error(f"Error in prediction: {e}")
